# Exercise 1

## Necessary imports and dataset loading

In [1]:
from nltk.corpus import wordnet as wn
import pandas as pd

In [2]:
# Reading the file
df = pd.read_csv('dataset\combined.csv')
df

,Word 1,Word 2,Human (mean)
0,love,sex,6.77
1,tiger,cat,7.35
2,tiger,tiger,10.00
3,book,paper,7.46
4,computer,keyboard,7.62
...,...,...,...
348,shower,flood,6.03
349,weather,forecast,8.34
350,disaster,area,6.25
351,governor,office,6.34


## Support functions

In [3]:
# Defining the function to get the max depth of the synset

def max_depth_synset(synset):
    return max(len(path) for path in synset.hypernym_paths())

In [4]:
# Defining the function to get the least common subsumer of two synsets

def LCS(syn1,syn2):
    path1 = syn1.hypernym_paths()
    path2 = syn2.hypernym_paths()
    result = []

    # We loop through the paths of the first synset
    for pat1 in path1:
        x = pat1
        x.reverse()
        found = False
        for syn in x:
            if(not found):
                # We loop through the paths of the second synset
                for pat2 in path2:
                    if(syn in pat2):
                        result.append(syn)
                        found = True
                        break
            else:
                break
    if not result:
            return None
    else:
         max_depth = max(max_depth_synset(s) for s in result)
         max_hypernym = max(s for s in result if max_depth_synset(s) == max_depth)
         return max_hypernym

In [19]:
def path_len(syn1,syn2):
    path1 = syn1.hypernym_paths()
    path2 = syn2.hypernym_paths()
    result_syn1 = []
    result_syn2 = []

    common_ancestor = LCS(syn1,syn2)

    if(common_ancestor is None):
        return None
    else:
        for pat1 in path1:
            if(common_ancestor in pat1):
                result_syn1.append(pat1.index(syn1)-pat1.index(common_ancestor))
        for pat2 in path2:
            if(common_ancestor in pat2):
                result_syn2.append(pat2.index(syn2)-pat2.index(common_ancestor))
        return min(result_syn1) + min(result_syn2)
  

## Wu Palmer

$$ LCS(s1, s2) = {2 * depth(LCS(s1,s2)) \over depth(s1) + depth(s2)} $$

In [13]:
def wu_palmer(word1,word2):
    # We get all the synsets of the two words
    synw1 = wn.synsets(word1)
    synw2 = wn.synsets(word2)
    result = []

    for syn1 in synw1:
        depht1 = max_depth_synset(syn1)
        for syn2 in synw2:
            depht2 = max_depth_synset(syn2)
            # Finding the least common subsumer of the two synsets
            common_ancestor = LCS(syn1,syn2)
            if(common_ancestor is None):
                break
            result.append( (2 * max_depth_synset(common_ancestor)) / ( depht1 + depht2 ) )
            
    return max(result)
    

In [14]:
print("WU Palmer between dog and cat:", wu_palmer('dog','cat'))

WU Palmer between dog and cat: 0.8571428571428571


## Shortest Path

$$ LCS(s1, s2) = 2 * depthMax - len(s1,s2) $$

In [39]:
depth_max = max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())
def shortest_path(word1,word2):
    synw1 = wn.synsets(word1)
    synw2 = wn.synsets(word2)
    result = []

    for syn1 in synw1:
        for syn2 in synw2:
            if path_len(syn1,syn2) is None:
                break
            elif (path_len(syn1,syn2) == 0):
                result.append(2 * depth_max)
            elif (path_len(syn1,syn2) == depth_max):
                result.append(0)
            else:
                result.append(path_len(syn1,syn2))
    return min(result)

In [42]:
print("Shortest Path between dog and cat:", shortest_path('dog','cat'))

Shortest Path between dog and cat: 4
